# Analyse Exploratoire des Performances Sectorielles

Ce notebook explore les performances des différents secteurs du marché (ETFs sectoriels) et leur comportement dans les différentes phases du cycle économique identifiées précédemment.

## Importation des bibliothèques

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import joblib

# Suppression des avertissements pour une meilleure lisibilité
warnings.filterwarnings('ignore')

# Configuration des visualisations
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Ajout du chemin du projet au PYTHONPATH

In [ ]:
# Ajout du répertoire racine du projet au path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)
print(f"Répertoire racine du projet: {project_root}")

## Importation des modules personnalisés

In [ ]:
from src.data.sector_data_collector import SectorDataCollector
from src.models.economic_cycle_classifier import EconomicCycleClassifier

## Collecte et chargement des données

In [ ]:
# Chemins des données prétraitées
sector_data_path = os.path.join(project_root, "data", "processed", "sector_data.csv")
phases_path = os.path.join(project_root, "data", "processed", "economic_phases.csv")

# Vérification si les données sectorielles existent, sinon les collecter
if not os.path.exists(sector_data_path):
    print("Collecte des données sectorielles...")
    # Création du répertoire si nécessaire
    os.makedirs(os.path.dirname(sector_data_path), exist_ok=True)
    
    # Collecte des données
    collector = SectorDataCollector()
    etf_data = collector.get_all_etf_data(start_date="2000-01-01")
    processed_data = collector.preprocess_data(etf_data)
    
    # Sauvegarde des données
    processed_data.to_csv(sector_data_path)
    print(f"Données sauvegardées dans {sector_data_path}")
else:
    print(f"Chargement des données depuis {sector_data_path}")

# Chargement des données sectorielles
sector_df = pd.read_csv(sector_data_path, index_col=0, parse_dates=True)
print(f"Données sectorielles chargées avec succès: {sector_df.shape[0]} observations et {sector_df.shape[1]} variables")

# Chargement des phases économiques
if os.path.exists(phases_path):
    phases_df = pd.read_csv(phases_path)
    phases_df['date'] = pd.to_datetime(phases_df['date'])
    phases_df.set_index('date', inplace=True)
    print(f"Phases économiques chargées avec succès: {phases_df.shape[0]} observations")
else:
    print("Phases économiques non trouvées. Veuillez d'abord exécuter le notebook 01_analyse_exploratoire_donnees_macro.ipynb")

## Aperçu des données sectorielles

In [ ]:
# Affichage des premières lignes
sector_df.head()

In [ ]:
# Liste des ETFs sectoriels et leur description
sector_etfs = {
    'XLY': 'Consommation discrétionnaire',
    'XLP': 'Consommation de base',
    'XLE': 'Énergie',
    'XLF': 'Finance',
    'XLV': 'Santé',
    'XLI': 'Industrie',
    'XLB': 'Matériaux',
    'XLK': 'Technologie',
    'XLU': 'Services publics',
    'XLRE': 'Immobilier',
    'XLC': 'Services de communication'
}

# Vérification des colonnes disponibles
available_sectors = []
for sector in sector_etfs.keys():
    if sector in sector_df.columns:
        available_sectors.append(sector)
        
print(f"Secteurs disponibles: {len(available_sectors)} sur {len(sector_etfs)}")
for sector in available_sectors:
    print(f"  - {sector}: {sector_etfs[sector]}")

## Extraction des prix et calcul des rendements

In [ ]:
# Extraction des prix de clôture ajustés
prices = sector_df[available_sectors].copy()

# Vérification des données de prix
print(f"Période couverte: {prices.index[0]} à {prices.index[-1]}")
print(f"Nombre d'observations: {len(prices)}")

# Affichage des premières lignes
prices.head()

In [ ]:
# Calcul des rendements mensuels
monthly_returns = prices.resample('M').last().pct_change() * 100
monthly_returns.index = monthly_returns.index.to_period('M')

# Affichage des rendements mensuels
monthly_returns.head()

## Performance historique des secteurs

In [ ]:
# Calcul de la performance cumulée
cumulative_returns = (1 + monthly_returns / 100).cumprod() - 1

# Visualisation de la performance cumulée
plt.figure(figsize=(14, 8))
for sector in available_sectors:
    plt.plot(cumulative_returns.index.to_timestamp(), cumulative_returns[sector] * 100, label=f"{sector} ({sector_etfs[sector]})")

plt.title('Performance cumulée des secteurs (en %)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Rendement cumulé (%)')
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Calcul des statistiques de performance annualisées
annual_stats = pd.DataFrame(index=available_sectors)

# Rendement annualisé
annual_stats['Rendement annualisé (%)'] = ((1 + monthly_returns / 100).prod() ** (12 / len(monthly_returns)) - 1) * 100

# Volatilité annualisée
annual_stats['Volatilité annualisée (%)'] = monthly_returns.std() * np.sqrt(12)

# Ratio de Sharpe (supposons un taux sans risque annualisé de 2%)
risk_free_rate = 0.02
annual_stats['Ratio de Sharpe'] = (annual_stats['Rendement annualisé (%)'] / 100 - risk_free_rate) / (annual_stats['Volatilité annualisée (%)'] / 100)

# Maximum drawdown
drawdowns = pd.DataFrame()
for sector in available_sectors:
    cumulative = (1 + monthly_returns[sector] / 100).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative / running_max) - 1
    drawdowns[sector] = drawdown
annual_stats['Maximum drawdown (%)'] = drawdowns.min() * 100

# Tri par rendement annualisé
annual_stats = annual_stats.sort_values('Rendement annualisé (%)', ascending=False)

# Ajout des noms complets des secteurs
annual_stats['Secteur'] = annual_stats.index.map(sector_etfs)
annual_stats = annual_stats[['Secteur', 'Rendement annualisé (%)', 'Volatilité annualisée (%)', 'Ratio de Sharpe', 'Maximum drawdown (%)']]

annual_stats

In [ ]:
# Visualisation des rendements vs volatilité
plt.figure(figsize=(12, 8))
plt.scatter(annual_stats['Volatilité annualisée (%)'], annual_stats['Rendement annualisé (%)'], s=100, alpha=0.7)

# Ajout des labels pour chaque point
for idx, row in annual_stats.iterrows():
    plt.annotate(idx, 
                 (row['Volatilité annualisée (%)'], row['Rendement annualisé (%)']),
                 xytext=(7, 0), 
                 textcoords='offset points',
                 fontsize=12)

plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='r', linestyle='--', alpha=0.3)

plt.title('Rendement vs Volatilité par secteur', fontsize=14)
plt.xlabel('Volatilité annualisée (%)')
plt.ylabel('Rendement annualisé (%)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Analyse de corrélation entre les secteurs

In [ ]:
# Calcul de la matrice de corrélation des rendements mensuels
corr_matrix = monthly_returns.corr()

# Visualisation de la matrice de corrélation
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Matrice de corrélation des rendements sectoriels', fontsize=14)
plt.tight_layout()
plt.show()

## Fusion des données sectorielles avec les phases économiques

In [ ]:
# Alignement des phases économiques avec les rendements mensuels
phases_monthly = phases_df.resample('M').last()
phases_monthly.index = phases_monthly.index.to_period('M')

# Fusion des données
merged_data = pd.merge(monthly_returns, phases_monthly, left_index=True, right_index=True, how='inner')

print(f"Données fusionnées: {merged_data.shape[0]} observations")
merged_data.head()

## Performance des secteurs par phase du cycle économique

In [ ]:
# Calcul des rendements moyens par phase et par secteur
phase_returns = merged_data.groupby('phase')[available_sectors].mean()

# Trier les phases dans l'ordre du cycle économique
cycle_order = ['Expansion', 'Surchauffe', 'Ralentissement', 'Récession', 'Reprise']
phase_returns = phase_returns.reindex(cycle_order)

# Visualisation des rendements moyens par phase
plt.figure(figsize=(14, 8))
ax = phase_returns.T.plot(kind='bar', figsize=(14, 8))
plt.title('Rendement mensuel moyen par secteur et par phase du cycle économique', fontsize=14)
plt.xlabel('Secteur')
plt.ylabel('Rendement mensuel moyen (%)')
plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.legend(title='Phase')
plt.tight_layout()
plt.show()

In [ ]:
# Visualisation heatmap des rendements par phase
plt.figure(figsize=(14, 10))
sns.heatmap(phase_returns, annot=True, cmap='RdYlGn', fmt='.2f', linewidths=0.5, center=0)
plt.title('Heatmap des rendements mensuels moyens par phase et secteur', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Calcul de la volatilité par phase et par secteur
phase_volatility = merged_data.groupby('phase')[available_sectors].std()
phase_volatility = phase_volatility.reindex(cycle_order)

# Visualisation de la volatilité par phase
plt.figure(figsize=(14, 10))
sns.heatmap(phase_volatility, annot=True, cmap='YlOrRd', fmt='.2f', linewidths=0.5)
plt.title('Volatilité mensuelle par phase et secteur', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Calcul du ratio rendement/risque par phase et par secteur
phase_sharpe = phase_returns / phase_volatility

# Visualisation du ratio rendement/risque par phase
plt.figure(figsize=(14, 10))
sns.heatmap(phase_sharpe, annot=True, cmap='RdYlGn', fmt='.2f', linewidths=0.5, center=0)
plt.title('Ratio rendement/risque par phase et secteur', fontsize=14)
plt.tight_layout()
plt.show()

## Analyse des meilleurs secteurs par phase

In [ ]:
# Sélection des 3 meilleurs secteurs par phase selon le rendement moyen
top_sectors = {}
worst_sectors = {}

for phase in cycle_order:
    # Meilleurs secteurs
    top = phase_returns.loc[phase].nlargest(3)
    top_sectors[phase] = [(sector, round(return_val, 2)) for sector, return_val in zip(top.index, top.values)]
    
    # Pires secteurs
    bottom = phase_returns.loc[phase].nsmallest(3)
    worst_sectors[phase] = [(sector, round(return_val, 2)) for sector, return_val in zip(bottom.index, bottom.values)]

# Affichage des résultats
print("Meilleurs secteurs par phase:")
for phase, sectors in top_sectors.items():
    print(f"\n{phase}:")
    for i, (sector, return_val) in enumerate(sectors):
        print(f"  {i+1}. {sector} ({sector_etfs[sector]}): {return_val}%")

print("\n" + "-"*50)

print("\nPires secteurs par phase:")
for phase, sectors in worst_sectors.items():
    print(f"\n{phase}:")
    for i, (sector, return_val) in enumerate(sectors):
        print(f"  {i+1}. {sector} ({sector_etfs[sector]}): {return_val}%")

In [ ]:
# Création d'un DataFrame pour les recommandations sectorielles par phase
recommendations = pd.DataFrame(index=cycle_order, columns=['Top 3 secteurs', 'À éviter'])

for phase in cycle_order:
    # Top 3 secteurs
    top3 = ", ".join([f"{s} ({sector_etfs[s]})" for s, _ in top_sectors[phase]])
    recommendations.loc[phase, 'Top 3 secteurs'] = top3
    
    # Secteurs à éviter
    avoid = ", ".join([f"{s} ({sector_etfs[s]})" for s, _ in worst_sectors[phase]])
    recommendations.loc[phase, 'À éviter'] = avoid

recommendations

## Création d'une stratégie simple de rotation sectorielle

In [ ]:
# Simulation d'une stratégie simple de rotation sectorielle basée sur les phases économiques
def simulate_rotation_strategy(returns, phases, top_n=3):
    """
    Simule une stratégie de rotation sectorielle simple.
    
    Args:
        returns (pd.DataFrame): Rendements mensuels des secteurs.
        phases (pd.Series): Phases économiques pour chaque mois.
        top_n (int): Nombre de secteurs à sélectionner par phase.
        
    Returns:
        pd.Series: Rendements mensuels de la stratégie.
    """
    # Fusion des données
    data = pd.merge(returns, phases, left_index=True, right_index=True, how='inner')
    
    # Initialisation des rendements de la stratégie
    strategy_returns = pd.Series(index=data.index, dtype=float)
    
    # Pour chaque mois
    for date, row in data.iterrows():
        phase = row['phase']
        
        # Sélection des top N secteurs pour cette phase (basée sur notre analyse)
        top_sectors_phase = [s for s, _ in top_sectors[phase][:top_n]]
        
        # Calcul du rendement moyen des secteurs sélectionnés pour ce mois
        selected_returns = row[top_sectors_phase]
        avg_return = selected_returns.mean()
        
        # Stockage du rendement de la stratégie
        strategy_returns[date] = avg_return
    
    return strategy_returns

# Simulation de la stratégie
strategy_returns = simulate_rotation_strategy(monthly_returns, phases_monthly['phase'])

# Calcul des rendements cumulés
cumulative_strategy = (1 + strategy_returns / 100).cumprod() - 1

# Calcul des rendements cumulés du S&P 500 (benchmark)
if 'SPY' in monthly_returns.columns:
    benchmark_returns = monthly_returns['SPY']
    cumulative_benchmark = (1 + benchmark_returns / 100).cumprod() - 1
else:
    # Utiliser la moyenne des secteurs comme approximation du marché
    benchmark_returns = monthly_returns[available_sectors].mean(axis=1)
    cumulative_benchmark = (1 + benchmark_returns / 100).cumprod() - 1

In [ ]:
# Visualisation des rendements cumulés
plt.figure(figsize=(14, 8))
plt.plot(cumulative_strategy.index.to_timestamp(), cumulative_strategy * 100, 'b-', linewidth=2, label='Stratégie de rotation sectorielle')
plt.plot(cumulative_benchmark.index.to_timestamp(), cumulative_benchmark * 100, 'r--', linewidth=2, label='Benchmark (S&P 500)')

plt.title('Performance de la stratégie de rotation sectorielle vs Benchmark', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Rendement cumulé (%)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Calcul des statistiques de performance
performance_stats = pd.DataFrame(index=['Stratégie', 'Benchmark'])

# Rendement annualisé
performance_stats['Rendement annualisé (%)'] = [
    ((1 + strategy_returns / 100).prod() ** (12 / len(strategy_returns)) - 1) * 100,
    ((1 + benchmark_returns / 100).prod() ** (12 / len(benchmark_returns)) - 1) * 100
]

# Volatilité annualisée
performance_stats['Volatilité annualisée (%)'] = [
    strategy_returns.std() * np.sqrt(12),
    benchmark_returns.std() * np.sqrt(12)
]

# Ratio de Sharpe
performance_stats['Ratio de Sharpe'] = [
    (performance_stats.loc['Stratégie', 'Rendement annualisé (%)'] / 100 - risk_free_rate) / (performance_stats.loc['Stratégie', 'Volatilité annualisée (%)'] / 100),
    (performance_stats.loc['Benchmark', 'Rendement annualisé (%)'] / 100 - risk_free_rate) / (performance_stats.loc['Benchmark', 'Volatilité annualisée (%)'] / 100)
]

# Maximum drawdown
for i, returns in enumerate([strategy_returns, benchmark_returns]):
    cumulative = (1 + returns / 100).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative / running_max) - 1
    max_dd = drawdown.min() * 100
    performance_stats.iloc[i, performance_stats.columns.get_loc('Maximum drawdown (%)')] = max_dd

performance_stats

## Analyse du momentum sectoriel

In [ ]:
# Calcul du momentum sur différentes périodes (3, 6, 12 mois)
momentum_periods = [3, 6, 12]
momentum_dfs = {}

for period in momentum_periods:
    # Calcul du momentum (rendement sur la période)
    momentum = monthly_returns.rolling(period).apply(lambda x: (1 + x/100).prod() - 1) * 100
    momentum_dfs[period] = momentum
    
    # Affichage des dernières valeurs
    print(f"\nMomentum sur {period} mois (dernier mois):")
    last_momentum = momentum.iloc[-1].sort_values(ascending=False)
    for sector, mom in last_momentum.items():
        if sector in sector_etfs:
            print(f"  {sector} ({sector_etfs[sector]}): {mom:.2f}%")

In [ ]:
# Visualisation du momentum à 6 mois
momentum_6m = momentum_dfs[6]
plt.figure(figsize=(14, 8))

for sector in available_sectors:
    plt.plot(momentum_6m.index.to_timestamp(), momentum_6m[sector], label=f"{sector} ({sector_etfs[sector]})")

plt.title('Momentum sur 6 mois par secteur', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Momentum sur 6 mois (%)')
plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Combinaison de l'analyse des cycles économiques et du momentum

In [ ]:
# Simulation d'une stratégie combinant l'analyse des cycles et le momentum
def simulate_combined_strategy(returns, phases, momentum_df, top_n=3, momentum_weight=0.5):
    """
    Simule une stratégie combinant l'analyse des cycles économiques et le momentum.
    
    Args:
        returns (pd.DataFrame): Rendements mensuels des secteurs.
        phases (pd.Series): Phases économiques pour chaque mois.
        momentum_df (pd.DataFrame): Momentum sectoriel.
        top_n (int): Nombre de secteurs à sélectionner.
        momentum_weight (float): Poids du momentum dans la sélection (0-1).
        
    Returns:
        pd.Series: Rendements mensuels de la stratégie.
    """
    # Fusion des données
    data = pd.merge(returns, phases, left_index=True, right_index=True, how='inner')
    
    # Initialisation des rendements de la stratégie
    strategy_returns = pd.Series(index=data.index, dtype=float)
    selected_sectors_history = {}
    
    # Pour chaque mois
    for date, row in data.iterrows():
        phase = row['phase']
        
        # Skip s'il n'y a pas de données de momentum disponibles
        if date not in momentum_df.index or momentum_df.loc[date].isna().all():
            continue
        
        # Scores de phase pour les secteurs (basés sur notre analyse)
        phase_scores = phase_returns.loc[phase].copy()
        
        # Normalisation des scores de phase (0-1)
        phase_scores = (phase_scores - phase_scores.min()) / (phase_scores.max() - phase_scores.min())
        
        # Scores de momentum
        momentum_scores = momentum_df.loc[date].copy()
        
        # Normalisation des scores de momentum (0-1)
        momentum_scores = (momentum_scores - momentum_scores.min()) / (momentum_scores.max() - momentum_scores.min())
        
        # Combinaison des scores
        combined_scores = phase_scores * (1 - momentum_weight) + momentum_scores * momentum_weight
        
        # Sélection des top N secteurs
        selected_sectors = combined_scores.nlargest(top_n).index.tolist()
        selected_sectors_history[date] = selected_sectors
        
        # Calcul du rendement moyen des secteurs sélectionnés pour ce mois
        selected_returns = row[selected_sectors]
        avg_return = selected_returns.mean()
        
        # Stockage du rendement de la stratégie
        strategy_returns[date] = avg_return
    
    return strategy_returns, selected_sectors_history

# Simulation de la stratégie combinée
combined_returns, selected_sectors = simulate_combined_strategy(
    monthly_returns, 
    phases_monthly['phase'], 
    momentum_dfs[6],  # Utilisation du momentum à 6 mois
    top_n=3, 
    momentum_weight=0.5  # 50% cycle, 50% momentum
)

# Calcul des rendements cumulés
cumulative_combined = (1 + combined_returns / 100).cumprod() - 1

In [ ]:
# Visualisation comparative des stratégies
plt.figure(figsize=(14, 8))
plt.plot(cumulative_combined.index.to_timestamp(), cumulative_combined * 100, 'g-', linewidth=2, label='Stratégie combinée (cycle + momentum)')
plt.plot(cumulative_strategy.index.to_timestamp(), cumulative_strategy * 100, 'b-', linewidth=2, label='Stratégie basée sur les cycles')
plt.plot(cumulative_benchmark.index.to_timestamp(), cumulative_benchmark * 100, 'r--', linewidth=2, label='Benchmark (S&P 500)')

plt.title('Comparaison des stratégies', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Rendement cumulé (%)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Calcul des statistiques de performance pour la stratégie combinée
performance_stats['Stratégie combinée'] = [
    ((1 + combined_returns / 100).prod() ** (12 / len(combined_returns)) - 1) * 100,  # Rendement annualisé
    combined_returns.std() * np.sqrt(12),  # Volatilité annualisée
    0,  # Placeholder pour le ratio de Sharpe
    0   # Placeholder pour le maximum drawdown
]

# Ratio de Sharpe
performance_stats.loc['Stratégie combinée', 'Ratio de Sharpe'] = (
    (performance_stats.loc['Stratégie combinée', 'Rendement annualisé (%)'] / 100 - risk_free_rate) / 
    (performance_stats.loc['Stratégie combinée', 'Volatilité annualisée (%)'] / 100)
)

# Maximum drawdown
cumulative = (1 + combined_returns / 100).cumprod()
running_max = cumulative.cummax()
drawdown = (cumulative / running_max) - 1
max_dd = drawdown.min() * 100
performance_stats.loc['Stratégie combinée', 'Maximum drawdown (%)'] = max_dd

performance_stats

## Analyse du turnover (rotation des secteurs)

In [ ]:
# Analyse du turnover des secteurs sélectionnés
selected_sectors_df = pd.DataFrame.from_dict(selected_sectors, orient='index')
selected_sectors_df.columns = [f'Secteur {i+1}' for i in range(selected_sectors_df.shape[1])]

# Calcul du turnover mensuel (nombre de changements de secteurs)
turnover = []
for i in range(1, len(selected_sectors_df)):
    prev_sectors = set(selected_sectors_df.iloc[i-1].values)
    curr_sectors = set(selected_sectors_df.iloc[i].values)
    changes = len(prev_sectors.symmetric_difference(curr_sectors)) / 2  # Division par 2 car chaque changement compte deux fois
    turnover.append(changes)

avg_turnover = np.mean(turnover)
print(f"Turnover mensuel moyen: {avg_turnover:.2f} secteurs sur {selected_sectors_df.shape[1]}")
print(f"Pourcentage de turnover: {avg_turnover/selected_sectors_df.shape[1]*100:.2f}%")

## Visualisation de l'allocation sectorielle au fil du temps

In [ ]:
# Création d'une matrice d'allocation (1 si le secteur est sélectionné, 0 sinon)
allocation_matrix = pd.DataFrame(index=selected_sectors_df.index, columns=available_sectors)
allocation_matrix = allocation_matrix.fillna(0)

for date, row in selected_sectors_df.iterrows():
    for sector in row.values:
        if sector in available_sectors:  # Vérification si le secteur est dans la liste
            allocation_matrix.loc[date, sector] = 1

# Visualisation de l'allocation sectorielle
plt.figure(figsize=(14, 8))
plt.imshow(allocation_matrix.T, aspect='auto', cmap='Blues', interpolation='none')

# Configuration des axes
plt.yticks(np.arange(len(available_sectors)), [f"{s} ({sector_etfs[s]})" for s in available_sectors])
plt.xticks(np.arange(0, len(allocation_matrix), 12), [date.strftime('%Y-%m') for date in allocation_matrix.index[::12]], rotation=90)

plt.title('Allocation sectorielle au fil du temps', fontsize=14)
plt.colorbar(label='Allocation')
plt.tight_layout()
plt.show()

## Conclusion

Dans cette analyse exploratoire des performances sectorielles, nous avons constaté que:

1. Les différents secteurs du marché ont des performances qui varient considérablement selon les phases du cycle économique.
2. Certains secteurs comme la Technologie (XLK) et la Consommation discrétionnaire (XLY) surperforment durant les phases d'Expansion et de Reprise.
3. Les secteurs défensifs comme les Services publics (XLU) et la Consommation de base (XLP) surperforment durant les phases de Ralentissement et de Récession.
4. Une stratégie de rotation sectorielle basée sur les phases du cycle économique et le momentum peut potentiellement surperformer le marché général.
5. La combinaison de l'analyse des cycles économiques et du momentum sectoriel semble être une approche prometteuse pour optimiser les rendements ajustés au risque.

Ces résultats confirment l'intérêt de développer une stratégie de rotation sectorielle dynamique basée sur l'identification des cycles économiques et l'analyse du momentum sectoriel.

In [ ]:
# Sauvegarde des secteurs recommandés par phase pour utilisation dans d'autres notebooks
recommendations_path = os.path.join(project_root, "data", "processed", "sector_recommendations.csv")
recommendations.to_csv(recommendations_path)
print(f"Recommandations sectorielles sauvegardées dans {recommendations_path}")

# Sauvegarde des performances des stratégies
performance_path = os.path.join(project_root, "data", "processed", "strategy_performance.csv")
performance_stats.to_csv(performance_path)
print(f"Performances des stratégies sauvegardées dans {performance_path}")